In [1]:
# Retrieve packages
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
# Define page
browser = Browser('chrome')
url = "https://basketball.realgm.com/nba/stats"
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()


In [3]:
# Retrieve Players
response = soup.find_all(class_="nowrap tablesaw-cell-persist")

players = []
url_player = []

for x in range(len(response)):
    players.append(response[x].text)

    text = response[x]
    href_pattern = r'<a href="([^"]+)">([^<]+)</a>'
    matches = re.findall(href_pattern, str(text))
    href_text = [match[0] for match in matches]
    href_text = href_text[0]
    href_text = "https://basketball.realgm.com/" + href_text
    url_player.append(href_text)



In [4]:
# Define page
browser = Browser('chrome')
url = url_player[0]
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# browser.quit()

In [5]:
# Find Current Team Function

def find_current_team(soup):
    response = soup.find_all(class_="half-column-left")
     
    current_team_pattern = r'<strong>Current Team:</strong>\s*<a href="[^"]+">([^<]+)</a>'
    match = re.search(current_team_pattern, str(response[0]))
    
    if match:
        current_team = match.group(1)
    else:
        current_team = None
    
    return current_team


In [6]:
# Find age Function

def find_age(soup):
    response = soup.find_all(class_="half-column-left")
        
    age_pattern = r'<strong>Born:</strong>\s*<a href="[^"]+">[^<]+</a>\s*\(([^)]+)\)'
    match = re.search(age_pattern, str(response[0]))
    
    if match:
        age = match.group(1)
    else:
        age = None
    
    return age

In [7]:
# Find height Function

def find_height(soup):
    response = soup.find_all(class_="half-column-left")
        
    height_pattern = r'<strong>Height:</strong>\s*([^<]+)'
    match = re.search(height_pattern, str(response[0]))
    
    if match:
        height = match.group(1).strip()
    else:
        height = None
    
    return height

In [8]:
# Find weight Function

def find_weight(soup):
    response = soup.find_all(class_="half-column-left")
        
    weight_pattern = r'<strong>Weight:</strong>\s*([^<]+)'
    match = re.search(weight_pattern, str(response[0]))
    
    if match:
        weight = match.group(1).strip()
    else:
        weight = None
    
    return weight

In [9]:
# Find current NBA status Function

def find_nba_status(soup):
    response = soup.find_all(class_="half-column-right")
        
    nba_status_pattern = r'<strong>Current NBA Status:</strong>\s*([^<]+)'
    match = re.search(nba_status_pattern, str(response[0]))
    
    if match:
        nba_status = match.group(1).strip()
    else:
        nba_status = None
    
    return nba_status



In [10]:
# Find draft entry Function

def find_draft_entry(soup):
    response = soup.find_all(class_="half-column-right")
        
    draft_entry_pattern = r'<strong>Draft Entry:</strong>\s*<a[^>]+>([^<]+)</a>'
    match = re.search(draft_entry_pattern, str(response[0]))
    
    if match:
        draft_entry = match.group(1).strip()
    else:
        draft_entry = None
    
    return draft_entry

In [92]:
# Retrieve score function

def find_scores(soup):
    response = soup.find_all(class_="per_game")
    
    year_overview = []
    gp_overview = []
    gs_overview = []
    min_overview = []
    pts_overview = []
    fgm_overview = []
    fga_overview = []
    fg_prc_overview = []
    three_pm_overview = []
    three_pa_overview = []
    three_p_prc_overview = []
    ftm_overview = []
    fta_overview = []
    ft_prc_overview = []
    off_overview = []
    deff_overview = []
    trb_overview = []
    ast_overview = []
    stl_overview = []
    blk_overview = []
    tov_overview = []
    pf_overview = []

    for x in range(len(response)-1):
        try:
            overview_scores = response[x+1].find_all('td')
            year = overview_scores[0].text
            year = year[:7]
            gp = overview_scores[2]
            gs = overview_scores[3]
            min = overview_scores[4]
            pts = overview_scores[5]
            fgm = overview_scores[6]
            fga = overview_scores[7]
            fg_prc = overview_scores[8]
            three_pm = overview_scores[9]
            three_pa = overview_scores[10]
            three_p_prc = overview_scores[11]
            ftm = overview_scores[12]
            fta = overview_scores[13]
            ft_prc = overview_scores[14]
            off = overview_scores[15]
            deff = overview_scores[16]
            trb = overview_scores[17]
            ast = overview_scores[18]
            stl = overview_scores[19]
            blk = overview_scores[20]
            tov = overview_scores[21]
            pf = overview_scores[22]

        except:
            pass

        if year[:2] == "20":
            year_overview.append(year)
            gp_overview.append(gp)
            gs_overview.append(gs)
            min_overview.append(min)
            pts_overview.append(pts)
            fgm_overview.append(fgm)
            fga_overview.append(fga)
            fg_prc_overview.append(fg_prc)
            three_pm_overview.append(three_pm)
            three_pa_overview.append(three_pa)
            three_p_prc_overview.append(three_p_prc)
            ftm_overview.append(ftm)
            fta_overview.append(fta)
            ft_prc_overview.append(ft_prc)
            off_overview.append(off)
            deff_overview.append(deff)
            trb_overview.append(trb)
            ast_overview.append(ast)
            stl_overview.append(stl)
            blk_overview.append(blk)
            tov_overview.append(tov)
            pf_overview.append(pf)
            
        else:
            pass
        
    find_scores_df = pd.DataFrame({"Year":year_overview,
                                   "GP":gp_overview,
                                   "GS": gs_overview,
                                   "MIN": min_overview,
                                   "PTS": pts_overview,
                                   "FGM": fgm_overview,
                                   "FGA": fga_overview,
                                   "FG%": fg_prc_overview,
                                   "3PM": three_pm_overview,
                                   "3PA": three_pa_overview,
                                   "3P%": three_p_prc_overview,
                                   "FTM": ftm_overview,
                                   "FTA": fta_overview,
                                   "FT%": ft_prc_overview,
                                   "OFF": off_overview,
                                   "DEF": deff_overview,
                                   "TRB": trb_overview,
                                   "AST": ast_overview,
                                   "STL": stl_overview,
                                   "BLK": blk_overview,
                                   "TOV": tov_overview,
                                   "PF": pf_overview})
    
    
    duplicate_mask = find_scores_df['Year'].duplicated()
    keep_mask = ~duplicate_mask
    find_scores_df = find_scores_df[keep_mask]

    find_scores_df['Year'] = find_scores_df['Year'].astype(str)

    rows_with_value = find_scores_df.index[find_scores_df['Year'] == '2022-23'].tolist()
    find_scores_df = find_scores_df.head(rows_with_value[0]+1)


    return find_scores_df



In [95]:
df = find_scores(soup)
df

,Year,GP,GS,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTA,FT%,OFF,DEF,TRB,AST,STL,BLK,TOV,PF
0,2016-17,[31],[31],[25.4],[20.23],[6.45],[13.84],[.466],[1.16],[3.16],...,[7.87],[.783],[1.97],[5.87],[7.84],[2.13],[0.87],[2.45],[3.77],[3.61]
1,2017-18,[63],[63],[30.4],[22.94],[8.10],[16.76],[.483],[1.05],[3.40],...,[7.41],[.769],[2.27],[8.68],[10.95],[3.16],[0.63],[1.76],[3.71],[3.32]
2,2018-19,[64],[64],[33.7],[27.52],[9.06],[18.73],[.484],[1.23],[4.11],...,[10.14],[.804],[2.50],[11.09],[13.59],[3.66],[0.72],[1.91],[3.53],[3.30]
3,2019-20,[51],[51],[29.5],[22.98],[7.49],[15.71],[.477],[1.12],[3.37],...,[8.53],[.807],[2.76],[8.86],[11.63],[2.98],[0.88],[1.27],[3.12],[3.41]
4,2020-21,[51],[51],[31.1],[28.45],[9.04],[17.63],[.513],[1.14],[3.02],...,[10.75],[.859],[2.22],[8.35],[10.57],[2.84],[0.98],[1.35],[3.12],[2.41]
5,2021-22,[68],[68],[33.8],[30.57],[9.79],[19.62],[.499],[1.37],[3.69],...,[11.81],[.814],[2.15],[9.56],[11.71],[4.18],[1.13],[1.46],[3.15],[2.66]
6,2022-23,[66],[66],[34.6],[33.08],[11.03],[20.12],[.548],[1.00],[3.03],...,[11.68],[.857],[1.71],[8.44],[10.15],[4.15],[1.00],[1.70],[3.42],[3.11]
